## 사용하는 패키지들

In [1]:
import json
import pandas as pd

import re

## 데이터 로드

In [2]:
with open('../Datasets/train.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [3]:
train_data = pd.DataFrame(json_data)
train_data = train_data.drop(['id', 'plylst_title', 'updt_date'], axis=1)
train_data.head()

,tags,songs,like_cnt
0,[락],"[525514, 129701, 383374, 562083, 297861, 13954...",71
1,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427...",1
2,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598...",17
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033...",33
4,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,...",9


In [4]:
with open('../Datasets/song_meta.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [5]:
song_data = pd.DataFrame(json_data)
song_data = song_data.drop(['album_name', 'song_gn_gnr_basket'], axis=1)
song_data.head()

,song_gn_dtl_gnr_basket,issue_date,album_id,artist_id_basket,song_name,artist_name_basket,id
0,[GN0901],20140512,2255639,[2727],Feelings,[Various Artists],0
1,"[GN1601, GN1606]",20080421,376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[Murray Perahia],1
2,[GN0901],20180518,4698747,[3361],Solsbury Hill (Remastered 2002),[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[Matoma],3
4,"[GN1802, GN1801]",20110824,2008470,[560160],그남자 그여자,[Jude Law],4


## 데이터 열 이름 변경

In [6]:
train_data.rename(columns={'songs':'song_id'}, inplace=True)
train_data.head()

,tags,song_id,like_cnt
0,[락],"[525514, 129701, 383374, 562083, 297861, 13954...",71
1,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427...",1
2,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598...",17
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033...",33
4,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,...",9


In [7]:
song_data.rename(columns={'id':'song_id', 'song_gn_dtl_gnr_basket': 'gnr'}, inplace=True)
song_data = song_data.astype({'issue_date':'int64'})
song_data.head()

,gnr,issue_date,album_id,artist_id_basket,song_name,artist_name_basket,song_id
0,[GN0901],20140512,2255639,[2727],Feelings,[Various Artists],0
1,"[GN1601, GN1606]",20080421,376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[Murray Perahia],1
2,[GN0901],20180518,4698747,[3361],Solsbury Hill (Remastered 2002),[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[Matoma],3
4,"[GN1802, GN1801]",20110824,2008470,[560160],그남자 그여자,[Jude Law],4


## 데이터 추출

- 500개의 플레이리스트 추출

In [8]:
train_data_sample = train_data[:500]

## 데이터 병합

In [18]:
train_data_explode = train_data_sample.explode('song_id', ignore_index=True)
train_data_explode.head()

,tags,song_id,like_cnt
0,[락],525514,71
1,[락],129701,71
2,[락],383374,71
3,[락],562083,71
4,[락],297861,71


In [19]:
train_dict = dict()

for i in range(len(train_data_explode)):
    song = train_data_explode['song_id'][i]
    tag = train_data_explode['tags'][i]
    
    if song in train_dict:
        for j in tag:
            train_dict[song].add(j)
    
    else:
        train_dict[song] = set(tag)
        
print(train_dict[157435])

{'스트레스해소', 'kpop', '댄스', '여자아이돌', '걸그룹댄스'}


In [20]:
train_data_explode.drop_duplicates(subset='song_id', keep='first',inplace=True)
train_data_explode.shape

(16674, 3)

In [21]:
for i in range(len(train_data_explode)):
    song = train_data_explode['song_id'].iloc[i]
    
    train_data_explode['tags'].iloc[i] = list(train_dict[song])

train_data_explode.head()

C:\Users\user\AppData\Local\Temp\ipykernel_14128\452334564.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_explode['tags'].iloc[i] = list(train_dict[song])


,tags,song_id,like_cnt
0,[락],525514,71
1,[락],129701,71
2,[락],383374,71
3,[락],562083,71
4,[락],297861,71


In [22]:
song_tag_appended = pd.merge(train_data_explode, song_data)
song_tag_appended = song_tag_appended.astype({'song_id':'int64'})
song_tag_appended.head()

,tags,song_id,like_cnt,gnr,issue_date,album_id,artist_id_basket,song_name,artist_name_basket
0,[락],525514,71,"[GN1402, GN1401]",20130506,2200223,[734201],Hey Little Girl,[The Sol]
1,[락],129701,71,"[GN0901, GN0902, GN1001]",20130917,2201802,[536907],Octagon,[Royal Bangs]
2,[락],383374,71,"[GN1012, GN1005, GN1001]",19911021,2216938,[166978],The Road,[Honeymoon Suite]
3,[락],562083,71,"[GN1013, GN0901, GN0902, GN1001]",20000919,43227,[19035],Honeymoon,[Phoenix]
4,[락],297861,71,"[GN1013, GN0901, GN0902, GN1001]",20050306,303657,[170117],High,[James Blunt]


In [23]:
song_tag_appended.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16674 entries, 0 to 16673
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tags                16674 non-null  object
 1   song_id             16674 non-null  int64 
 2   like_cnt            16674 non-null  int64 
 3   gnr                 16674 non-null  object
 4   issue_date          16674 non-null  int64 
 5   album_id            16674 non-null  int64 
 6   artist_id_basket    16674 non-null  object
 7   song_name           16674 non-null  object
 8   artist_name_basket  16674 non-null  object
dtypes: int64(4), object(5)
memory usage: 1.3+ MB


## 데이터 전처리

## 태그 세트 생성

In [24]:
def make_tag_set(data):
    tag_set = set()

    for i in range(len(data)):
        tag = data['tags'][i]
    
        for j in tag:
            tag_set.add(j)

    return tag_set

In [25]:
tag_set = make_tag_set(train_data_sample)
print(tag_set)

{'피트니스', '눈오는날', '다이어트', '헬스장', '재즈', '댄스신에', '밴드', '분위기좋은', '둠칫둠칫', '남녀', '엑소', '잔잔하게', '들어봤을', '달달한', 'futurebass', '따스한', '격렬함', '기분전환', '초록빛', '샤워', '연말파티', '고음병', '팝', '자연주의', '질리지않는', '환상적인', '힐링피아노연주곡', '유니크', '자연', 'GZ', '에센셜', 'Chillout', '우아한', '헤이즈', '추위', '칠링', '이루마', 'Duet', '아이돌', '소원', '태교클래식', '어쿠스틱', 'ksks', 'RNBSOUL', 'BTS', '히피', '퀸덤원곡모음', '중국음악', '펍', '눈', 'M에센셜', 'EDM', '느낌있는', '조용한', '힙한', '추천노래', '라운지', '내가좋아하는음악트랙', '매장', '댄스음악', '스윙', '랩소디인블루', '토요명화', 'Feat', '아미라면', '뺏고싶다', '목록', '피아노', 'Top100에', '뉴페이스', '퓨전재즈', '썸타는중', '자기전', '버스', '가을', '여자가수', '힘든날', '나만알기아까운노래', '파티', '아기동요', '진원', '다중음격', '다짐', '메탈', '까페', 'Chill', '아쉬움', '쓸쓸한', '키즈동화', '집', '밝은', '섹시', '스테파니', '미세먼지', 'RnB', '이슬', '이소라', '공부하며듣는', '최신', '릴랙스요가클래식', 'Metal', '가요', '파란', 'ASMR', '광고', '비와이', '국외', '외로움을', '김광민', '한번쯤', '신맛', '쓸쓸함', '지칠때', '업데이트중', 'funk', '힘내', '여름방학', '몽글', '내적댄스폭발', '필라테스', '9살', '타이틀곡', '소망', '8월', '비_오는날', '불면증', '고속도로', '인생곡들만', 'EarlySoul', '잔디', '성탄절', '후회', '기리보이

## 태그 dict 생성 함수

## 영어 대소문자 통일

- 소문자로 통일

In [26]:
def make_dict_lower(tag_set):
    tag_dict = dict()
    
    for i in tag_set:
        if not i in tag_dict:
            tag_dict[i] = i.lower()
    
    return tag_dict

In [27]:
tag_lower_dict = make_dict_lower(tag_set)
print(tag_lower_dict)

{'피트니스': '피트니스', '눈오는날': '눈오는날', '다이어트': '다이어트', '헬스장': '헬스장', '재즈': '재즈', '댄스신에': '댄스신에', '밴드': '밴드', '분위기좋은': '분위기좋은', '둠칫둠칫': '둠칫둠칫', '남녀': '남녀', '엑소': '엑소', '잔잔하게': '잔잔하게', '들어봤을': '들어봤을', '달달한': '달달한', 'futurebass': 'futurebass', '따스한': '따스한', '격렬함': '격렬함', '기분전환': '기분전환', '초록빛': '초록빛', '샤워': '샤워', '연말파티': '연말파티', '고음병': '고음병', '팝': '팝', '자연주의': '자연주의', '질리지않는': '질리지않는', '환상적인': '환상적인', '힐링피아노연주곡': '힐링피아노연주곡', '유니크': '유니크', '자연': '자연', 'GZ': 'gz', '에센셜': '에센셜', 'Chillout': 'chillout', '우아한': '우아한', '헤이즈': '헤이즈', '추위': '추위', '칠링': '칠링', '이루마': '이루마', 'Duet': 'duet', '아이돌': '아이돌', '소원': '소원', '태교클래식': '태교클래식', '어쿠스틱': '어쿠스틱', 'ksks': 'ksks', 'RNBSOUL': 'rnbsoul', 'BTS': 'bts', '히피': '히피', '퀸덤원곡모음': '퀸덤원곡모음', '중국음악': '중국음악', '펍': '펍', '눈': '눈', 'M에센셜': 'm에센셜', 'EDM': 'edm', '느낌있는': '느낌있는', '조용한': '조용한', '힙한': '힙한', '추천노래': '추천노래', '라운지': '라운지', '내가좋아하는음악트랙': '내가좋아하는음악트랙', '매장': '매장', '댄스음악': '댄스음악', '스윙': '스윙', '랩소디인블루': '랩소디인블루', '토요명화': '토요명화', 'Feat': 'feat', '아미라면': '아미라면', '뺏고싶다

## 특수문자 제거

In [28]:
def make_dict_special(tag_set):
    tag_dict = dict()
    
    for i in tag_set:
        if not i in tag_dict:
            # 아래를 제외한 문자는 ''로 대체한다
            # \uAC00-WuD7A3 : 모든 한글 음절
            # 0-9 : 숫자
            # a-zA-Z : 모든 영어
            tag_dict[i] = re.sub(r'[^\uAC00-\uD7A30-9a-zA-Z]', '', i)
    
    return tag_dict

In [29]:
tag_special_dict = make_dict_special(tag_set)

# 현재 태그들은 특수문자가 '_'만 존재한다
print(tag_special_dict)

{'피트니스': '피트니스', '눈오는날': '눈오는날', '다이어트': '다이어트', '헬스장': '헬스장', '재즈': '재즈', '댄스신에': '댄스신에', '밴드': '밴드', '분위기좋은': '분위기좋은', '둠칫둠칫': '둠칫둠칫', '남녀': '남녀', '엑소': '엑소', '잔잔하게': '잔잔하게', '들어봤을': '들어봤을', '달달한': '달달한', 'futurebass': 'futurebass', '따스한': '따스한', '격렬함': '격렬함', '기분전환': '기분전환', '초록빛': '초록빛', '샤워': '샤워', '연말파티': '연말파티', '고음병': '고음병', '팝': '팝', '자연주의': '자연주의', '질리지않는': '질리지않는', '환상적인': '환상적인', '힐링피아노연주곡': '힐링피아노연주곡', '유니크': '유니크', '자연': '자연', 'GZ': 'GZ', '에센셜': '에센셜', 'Chillout': 'Chillout', '우아한': '우아한', '헤이즈': '헤이즈', '추위': '추위', '칠링': '칠링', '이루마': '이루마', 'Duet': 'Duet', '아이돌': '아이돌', '소원': '소원', '태교클래식': '태교클래식', '어쿠스틱': '어쿠스틱', 'ksks': 'ksks', 'RNBSOUL': 'RNBSOUL', 'BTS': 'BTS', '히피': '히피', '퀸덤원곡모음': '퀸덤원곡모음', '중국음악': '중국음악', '펍': '펍', '눈': '눈', 'M에센셜': 'M에센셜', 'EDM': 'EDM', '느낌있는': '느낌있는', '조용한': '조용한', '힙한': '힙한', '추천노래': '추천노래', '라운지': '라운지', '내가좋아하는음악트랙': '내가좋아하는음악트랙', '매장': '매장', '댄스음악': '댄스음악', '스윙': '스윙', '랩소디인블루': '랩소디인블루', '토요명화': '토요명화', 'Feat': 'Feat', '아미라면': '아미라면', '뺏고싶다

## Stopword 제거

- 현재 태그는 단순 단어나 띄어쓰기가 제거된 단어의 조합으로 구성
- 임의의 Stopword를 선정할 필요가 존재

In [30]:
# 설정 필요
stopwords = {'날려버려'}

In [31]:
def make_dict_stop(tag_set, stopwords):
    tag_dict = dict()
    
    for i in tag_set:
        if not i in tag_dict:
            # 빈 칸으로 만든 뒤 다른 함수로 제거할 예정
            if i in stopwords:
                tag_dict[i] = ''
            else:
                tag_dict[i] = i
                
    return tag_dict

In [32]:
tag_stop_dict = make_dict_stop(tag_set, stopwords)
print(tag_stop_dict)

{'피트니스': '피트니스', '눈오는날': '눈오는날', '다이어트': '다이어트', '헬스장': '헬스장', '재즈': '재즈', '댄스신에': '댄스신에', '밴드': '밴드', '분위기좋은': '분위기좋은', '둠칫둠칫': '둠칫둠칫', '남녀': '남녀', '엑소': '엑소', '잔잔하게': '잔잔하게', '들어봤을': '들어봤을', '달달한': '달달한', 'futurebass': 'futurebass', '따스한': '따스한', '격렬함': '격렬함', '기분전환': '기분전환', '초록빛': '초록빛', '샤워': '샤워', '연말파티': '연말파티', '고음병': '고음병', '팝': '팝', '자연주의': '자연주의', '질리지않는': '질리지않는', '환상적인': '환상적인', '힐링피아노연주곡': '힐링피아노연주곡', '유니크': '유니크', '자연': '자연', 'GZ': 'GZ', '에센셜': '에센셜', 'Chillout': 'Chillout', '우아한': '우아한', '헤이즈': '헤이즈', '추위': '추위', '칠링': '칠링', '이루마': '이루마', 'Duet': 'Duet', '아이돌': '아이돌', '소원': '소원', '태교클래식': '태교클래식', '어쿠스틱': '어쿠스틱', 'ksks': 'ksks', 'RNBSOUL': 'RNBSOUL', 'BTS': 'BTS', '히피': '히피', '퀸덤원곡모음': '퀸덤원곡모음', '중국음악': '중국음악', '펍': '펍', '눈': '눈', 'M에센셜': 'M에센셜', 'EDM': 'EDM', '느낌있는': '느낌있는', '조용한': '조용한', '힙한': '힙한', '추천노래': '추천노래', '라운지': '라운지', '내가좋아하는음악트랙': '내가좋아하는음악트랙', '매장': '매장', '댄스음악': '댄스음악', '스윙': '스윙', '랩소디인블루': '랩소디인블루', '토요명화': '토요명화', 'Feat': 'Feat', '아미라면': '아미라면', '뺏고싶다

## Stemming 제거

- 한글에서 동일한 의미여도 표현이 다른 경우를 자동적으로 찾는 것은 불가능하다
- 임의로 동일한 의미의 단어 묶음을 만들 필요가 있다

In [33]:
# 설정 필요
stemmings = {
    '락' : {'락', '록'}
}

In [34]:
def make_dict_stem(tag_set, stemmings):
    tag_dict = dict()
    
    for i in tag_set:
        if not i in tag_dict:
            
            # stemming으로 설정한 단어가 존재하는지 확인
            for j in stemmings:
                if i in stemmings[j]:
                    tag_dict[i] = j
                    break
            
            # 존재하지 않는 경우
            if not i in tag_dict:
                tag_dict[i] = i
    
    return tag_dict

In [35]:
tag_stem_dict = make_dict_stem(tag_set, stemmings)
print(tag_stem_dict)

{'피트니스': '피트니스', '눈오는날': '눈오는날', '다이어트': '다이어트', '헬스장': '헬스장', '재즈': '재즈', '댄스신에': '댄스신에', '밴드': '밴드', '분위기좋은': '분위기좋은', '둠칫둠칫': '둠칫둠칫', '남녀': '남녀', '엑소': '엑소', '잔잔하게': '잔잔하게', '들어봤을': '들어봤을', '달달한': '달달한', 'futurebass': 'futurebass', '따스한': '따스한', '격렬함': '격렬함', '기분전환': '기분전환', '초록빛': '초록빛', '샤워': '샤워', '연말파티': '연말파티', '고음병': '고음병', '팝': '팝', '자연주의': '자연주의', '질리지않는': '질리지않는', '환상적인': '환상적인', '힐링피아노연주곡': '힐링피아노연주곡', '유니크': '유니크', '자연': '자연', 'GZ': 'GZ', '에센셜': '에센셜', 'Chillout': 'Chillout', '우아한': '우아한', '헤이즈': '헤이즈', '추위': '추위', '칠링': '칠링', '이루마': '이루마', 'Duet': 'Duet', '아이돌': '아이돌', '소원': '소원', '태교클래식': '태교클래식', '어쿠스틱': '어쿠스틱', 'ksks': 'ksks', 'RNBSOUL': 'RNBSOUL', 'BTS': 'BTS', '히피': '히피', '퀸덤원곡모음': '퀸덤원곡모음', '중국음악': '중국음악', '펍': '펍', '눈': '눈', 'M에센셜': 'M에센셜', 'EDM': 'EDM', '느낌있는': '느낌있는', '조용한': '조용한', '힙한': '힙한', '추천노래': '추천노래', '라운지': '라운지', '내가좋아하는음악트랙': '내가좋아하는음악트랙', '매장': '매장', '댄스음악': '댄스음악', '스윙': '스윙', '랩소디인블루': '랩소디인블루', '토요명화': '토요명화', 'Feat': 'Feat', '아미라면': '아미라면', '뺏고싶다

## 태그 dict 적용 함수

In [36]:
def apply_tag_dict(data, tag_dict):
    apply_data = data.copy()
    
    for i in range(len(apply_data)):
        tag = apply_data['tags'][i]
        apply_tag = []
        
        for j in tag:
            if not tag_dict[j] in apply_tag and tag_dict[j] != "":
                apply_tag.append(tag_dict[j])
        
        apply_data['tags'][i] = apply_tag

    return apply_data

In [37]:
stop_data = apply_tag_dict(train_data_sample, tag_stop_dict)
stop_data.head()

C:\Users\user\AppData\Local\Temp\ipykernel_14128\2377676666.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apply_data['tags'][i] = apply_tag


,tags,song_id,like_cnt
0,[락],"[525514, 129701, 383374, 562083, 297861, 13954...",71
1,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427...",1
2,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598...",17
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033...",33
4,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,...",9


In [38]:
# stopword가 제거됨
stop_tag_set = make_tag_set(stop_data)
print(stop_tag_set)

{'피트니스', '눈오는날', '다이어트', '헬스장', '재즈', '댄스신에', '밴드', '분위기좋은', '둠칫둠칫', '남녀', '엑소', '잔잔하게', '들어봤을', '달달한', 'futurebass', '따스한', '격렬함', '기분전환', '초록빛', '샤워', '연말파티', '고음병', '팝', '자연주의', '질리지않는', '환상적인', '힐링피아노연주곡', '유니크', '자연', 'GZ', '에센셜', 'Chillout', '우아한', '헤이즈', '추위', '칠링', '이루마', 'Duet', '아이돌', '소원', '태교클래식', '어쿠스틱', 'ksks', 'RNBSOUL', 'BTS', '히피', '퀸덤원곡모음', '중국음악', '펍', '눈', 'M에센셜', 'EDM', '느낌있는', '조용한', '힙한', '추천노래', '라운지', '내가좋아하는음악트랙', '매장', '댄스음악', '스윙', '랩소디인블루', '토요명화', 'Feat', '아미라면', '뺏고싶다', '목록', '피아노', 'Top100에', '뉴페이스', '퓨전재즈', '썸타는중', '자기전', '버스', '가을', '여자가수', '힘든날', '나만알기아까운노래', '파티', '아기동요', '진원', '다중음격', '다짐', '메탈', '까페', 'Chill', '아쉬움', '쓸쓸한', '키즈동화', '집', '밝은', '섹시', '스테파니', '미세먼지', 'RnB', '이슬', '이소라', '공부하며듣는', '최신', '릴랙스요가클래식', 'Metal', '가요', '파란', 'ASMR', '광고', '비와이', '국외', '외로움을', '김광민', '한번쯤', '신맛', '쓸쓸함', '지칠때', '업데이트중', 'funk', '힘내', '여름방학', '몽글', '내적댄스폭발', '필라테스', '9살', '타이틀곡', '소망', '8월', '비_오는날', '불면증', '고속도로', '인생곡들만', 'EarlySoul', '잔디', '성탄절', '후회', '기리보이

## 태그 불균형 데이터

## 불균형 정도 측정

- 한 태그를 가지는 음악의 최대 개수, 한 태그를 가지는 음악의 개수를 추출
- 가중치 부여에 관해서는 논의가 필요

In [39]:
def make_song_num_dict(data):
    song_ids = dict()
    song_num = dict()
    max_num = 0
    
    for i in range(len(data)):
        songs = data['song_id'][i]
        tags = data['tags'][i]
        
        for j in tags:
            if not j in song_ids:
                song_ids[j] = set(songs)
            
            else:
                song_ids[j].update(songs)
    
    for i in song_ids:
        song_num[i] = len(song_ids[i])
        
        max_num = max(song_num[i], max_num)
    
    return song_num, max_num

In [40]:
song_num_dict, song_num_max = make_song_num_dict(train_data_sample)
print(song_num_max)
print(song_num_dict)

2568
{'락': 392, '추억': 1263, '회상': 853, '까페': 743, '잔잔한': 1753, '연말': 71, '눈오는날': 38, '캐럴': 71, '분위기': 598, '따듯한': 38, '크리스마스캐럴': 38, '겨울노래': 79, '크리스마스': 129, '겨울왕국': 38, '크리스마스송': 38, '댄스': 850, '운동': 655, '드라이브': 2214, 'Pop': 893, '트로피컬하우스': 30, '힐링': 2480, '기분전환': 2484, '2017': 30, '팝': 1075, '트렌드': 30, '일렉': 300, '짝사랑': 52, '취향저격': 548, '슬픔': 731, '고백': 85, '사랑': 1629, '이별': 787, '일렉트로니카': 167, '포크': 134, '메탈': 74, '인디': 1092, '록': 54, 'Metal': 34, '이일우': 15, 'M에센셜': 15, 'Rock': 119, 'kpop': 157, '걸그룹댄스': 157, '스트레스해소': 166, '새해': 45, '여행': 1065, '프로필음악': 45, '카카오톡': 45, '소원': 45, '프로필': 45, '소망': 45, '다짐': 45, '카톡': 45, '듣고': 21, '우울': 294, '이거': 21, '힘내': 90, '힙합': 1316, '느낌있는': 425, '밤': 1498, '새벽': 1603, 'RnB': 310, '감각적인': 217, '국내': 66, '그루브한': 66, '가을': 569, '재즈': 401, '감성': 1871, '질리지않는': 31, '나만알고싶은': 79, '봄': 432, '설렘': 940, '비오는날': 1087, '누군가생각날때': 18, '스밍': 16, '목록': 16, '폐막식': 16, '올림픽': 16, '엑소': 16, '조용히': 44, '혼자': 190, '또는': 44, '새벽감성': 54, '고민': 44, '맥주한잔': 44, '카

## 장르 데이터

## 불균형 데이터 처리

In [48]:
def make_genre_num_dict(appended_data):
    song_ids = dict()
    song_num = dict()
    max_num = 0
    
    for i in range(len(appended_data)):
        song = appended_data['song_id'][i]
        gnrs = appended_data['gnr'][i]
        
        for j in gnrs:
            if not j in song_ids:
                song_ids[j] = {song}
            
            else:
                song_ids[j].add(song)
    
    for i in song_ids:
        song_num[i] = len(song_ids[i])
        
        max_num = max(song_num[i], max_num)
    
    return song_num, max_num

In [49]:
genre_num_dict, genre_num_max = make_genre_num_dict(song_tag_appended)
print(genre_num_max)
print(genre_num_dict)

3049
{'GN1402': 102, 'GN1401': 180, 'GN0901': 1783, 'GN0902': 353, 'GN1001': 953, 'GN1012': 80, 'GN1005': 22, 'GN1013': 276, 'GN1003': 102, 'GN1304': 218, 'GN1301': 766, 'GN1302': 605, 'GN0904': 27, 'GN1912': 138, 'GN1904': 25, 'GN1901': 143, 'GN1102': 361, 'GN1101': 942, 'GN0101': 3049, 'GN0103': 175, 'GN0601': 1118, 'GN0605': 266, 'GN0104': 612, 'GN0508': 101, 'GN0501': 1731, 'GN0503': 562, 'GN0805': 514, 'GN0502': 468, 'GN0801': 670, 'GN0509': 1506, 'GN0105': 1710, 'GN0606': 680, 'GN1701': 657, 'GN1706': 88, 'GN1803': 63, 'GN1801': 661, 'GN1608': 31, 'GN1601': 439, 'GN1501': 1004, 'GN1504': 600, 'GN2601': 175, 'GN2603': 91, 'GN2602': 54, 'GN1708': 12, 'GN1502': 71, 'GN0401': 974, 'GN0403': 626, 'GN0402': 250, 'GN0303': 756, 'GN0301': 1363, 'GN1703': 107, 'GN0504': 70, 'GN0908': 303, 'GN1509': 33, 'GN2207': 25, 'GN1506': 47, 'GN0907': 33, 'GN1702': 117, 'GN0506': 214, 'GN0702': 10, 'GN0708': 11, 'GN0701': 24, 'GN2502': 630, 'GN2506': 793, 'GN2501': 1358, 'GN0203': 133, 'GN0201': 1267

## Embedding

In [51]:
song_tag_appended['gnr_literal'] = song_tag_appended['gnr'].apply(lambda x : (' ').join(x))
song_tag_appended.head()

,tags,song_id,like_cnt,gnr,issue_date,album_id,artist_id_basket,song_name,artist_name_basket,gnr_literal
0,[락],525514,71,"[GN1402, GN1401]",20130506,2200223,[734201],Hey Little Girl,[The Sol],GN1402 GN1401
1,[락],129701,71,"[GN0901, GN0902, GN1001]",20130917,2201802,[536907],Octagon,[Royal Bangs],GN0901 GN0902 GN1001
2,[락],383374,71,"[GN1012, GN1005, GN1001]",19911021,2216938,[166978],The Road,[Honeymoon Suite],GN1012 GN1005 GN1001
3,[락],562083,71,"[GN1013, GN0901, GN0902, GN1001]",20000919,43227,[19035],Honeymoon,[Phoenix],GN1013 GN0901 GN0902 GN1001
4,[락],297861,71,"[GN1013, GN0901, GN0902, GN1001]",20050306,303657,[170117],High,[James Blunt],GN1013 GN0901 GN0902 GN1001


## TF-IDF

## 벡터 평균

## 공통

## 평점에 따른 가중치